In [30]:
import requests
import json
import pandas as pd

# Find all pages id from 25 pages with each page contains 25 lawyer information
all_pages = []
all_ids = []

url = "https://az-ae-app-fal-prod-webservice.azurewebsites.net/api/lawyer/"
for i in range(1,10000):
    pages = requests.post(url,json={"lastName":"","otherName":"","suburb":"","region":"","accreditedSpecialist":"","page":i,"pageSize":100})
    pages_data = pages.json()['results']
    if len(pages_data) == 0:     
        break
    for id in range(len(pages_data)):
        all_ids.append(pages_data[id]['id'])
    
all_unique_ids = list(set(all_ids))

In [35]:
all_jsons = []
for i in range(len(all_unique_ids)):
    url = f"https://az-ae-app-fal-prod-webservice.azurewebsites.net/api/lawyer/{all_ids[i]}"
    response = requests.get(url)
    all_jsons.append(response.json())

In [95]:
firm_name = []
firm_phone = []
firm_fax = []
firm_languages = []
firm_region = []
firm_email = []

for i in range(len(all_unique_ids)):
    try:
        firm_name.append(all_jsons[i]['firstName'] + ' ' + all_jsons[i]['middleName'] + ' ' + all_jsons[i]['lastName'])
        firm_phone.append(all_jsons[i]['firmPhone'])
        firm_fax.append(all_jsons[i]['firmFax'])
        firm_email.append(all_jsons[i]['firmEmail'])
        firm_languages.append(all_jsons[i]['languages'])
        if all_jsons[i]['streetAddress'] is None:
            firm_region.append('')
        else:
            firm_region.append(all_jsons[i]['streetAddress']['suburb'])
        
    except TypeError:
        pass

In [98]:
data = {
    'Firm Name': firm_name,
    'Firm Email': firm_email,
    'Firm Phone': firm_phone,
    'Firm Fax': firm_fax,
    'Firm Region': firm_region,
    'Firm Languages': firm_languages
}

final_csv = pd.DataFrame(data)

In [99]:
for i in range(len(final_csv['Firm Languages'])):
    if final_csv['Firm Languages'][i] is None:
        final_csv['Firm Languages'][i] = []
        
for i in range(len(final_csv)):
    if len(final_csv['Firm Languages'][i]) != 0:
        final_csv['Firm Languages'][i] = ', '.join([str(_) for _ in final_csv['Firm Languages'][i]])
        
    else:
        final_csv['Firm Languages'][i] = ''

In [101]:
final_csv.drop_duplicates().to_csv('NSW Law Firms.csv', index=False)